# functools

> The [functools](https://docs.python.org/3.8/library/functools.html) module is for higher-order functions: functions that act on or return other functions. In general, any callable object can be treated as a function for the purposes of this module.

***

## functools.cached_property

`cached_property` is applied to a method of a class that is normally expensive to compute but **also doesn't change throughout the lifetime of the object**. The function turns a method into a property and caches the results the first time the method is called. Every subsequent time the method is called the cached results are returned.

In [6]:
from functools import cached_property

In [25]:
class LongList:
    
    def __init__(self, data):
        self._data = data
        
    @cached_property
    def maximum(self):
        print('Only prints first time')
        return max(self._data)
    
    @cached_property
    def minimum(self):
        print('Only prints first time')
        return min(self._data)

In [39]:
import random

ls = LongList([random.randint(5, 500) for _ in range(1_000_000)])

print(ls.maximum, ls.minimum, sep='\t', end='\n\n')
print(ls.maximum, ls.minimum, sep='\t')

# Even when you change the underlying data structure
# that minimum and maximum work on
ls._data = [random.randint(100, 300) for _ in range(1_000_000)]
# minimum and maximum don't recompute 
print(ls.maximum, ls.minimum, sep='\t')

Only prints first time
Only prints first time
500	5

500	5
500	5


## functools.cmp_to_key

Transforms a comparison function into a [key function](https://docs.python.org/3.8/glossary.html#term-key-function). \
A comparison function is one that takes in two arguments, call them `a` and `b`, and returns -1 if a < b, 0 if a == b, or 1 if a > b.

In [ ]:
def compare()